# 3. 컬럼별 최적 정규화 과정

In [1]:
#최적화 과정에서 사용했던 패키지
import pandas as pd
import numpy as np
import scipy as sp

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk

import datetime as dt
from patsy import dmatrix

import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes(palette="muted")

from statsmodels.stats.outliers_influence import variance_inflation_factor

%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
from taxi_pakage import *

C:\Users\ledes\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
import folium

In [3]:
#아웃라이어 확인
def pick_dropoff(data) :
    """
    Install & import folium
    Find out where it was picked up and dropped off
    """
    
    pick_lat = data["pickup_latitude"]
    pick_lon = data["pickup_longitude"]
    dropoff_lat = data["dropoff_latitude"]
    dropoff_lon = data["dropoff_longitude"]
    info = data["id"]
    
    
    center = [40.745219,-73.9935417] # lat, long
    map_osm = folium.Map(location = center, zoom_start = 10, tiles = 'Stamen toner') # Stamen toner - 빠른 로딩 가능
    
    #승차 지점, 파랑으로 표시
    for lat, long, info_ in zip(pick_lat, pick_lon, info) :
        folium.Marker([lat, long], popup = str(info_) , icon = folium.Icon(color = "blue")).add_to(map_osm)
    
    #하차 지점, 빨강으로 표시
    for lat_, long_, info_ in zip(dropoff_lat, dropoff_lon, info) :
        folium.Marker([lat_, long_], popup = str(info_) , icon = folium.Icon(color = "red")).add_to(map_osm)
    
    return map_osm
    
    

In [4]:
#전처리 된 데이터 가져오기
taxi = pd.read_csv("edited_taxi.csv")

In [5]:
train = pd.read_csv("train.csv", usecols = ["pickup_datetime", "dropoff_datetime"])

(kaggle에서 pickup_hour가 feature로써 중요하다는 커널을 발견)

In [6]:
#pickup_hour, dropoff_hour 추가
train["pickup_hour"] = pd.to_datetime(train["pickup_datetime"]).dt.hour
train["dropoff_hour"] = pd.to_datetime(train["dropoff_datetime"]).dt.hour

taxi["pickup_hour"] = train["pickup_hour"]
taxi["dropoff_hour"] = train["dropoff_hour"]

trip_duration = taxi["trip_duration"]

taxi["trip_duration"] = trip_duration

(외부데이터 추가 - weahter event)  
[날짜들에 대한 설명](https://www.weather.gov/okx/stormevents)

In [7]:
#extreme_weather data 추가 입력
weather_event = ['20160110', '20160113', '20160117', '20160123',
                 '20160205', '20160208', '20160215', '20160216',
                 '20160224', '20160225', '20160314', '20160315',
                 '20160328', '20160329', '20160403', '20160404',
                 '20160530', '20160628']
weather_event = pd.Series(pd.to_datetime(weather_event, format = '%Y%m%d')).dt.date
weather_event = weather_event.astype('<U32')
weather_event = list(weather_event)

train["y-m-d"] = pd.to_datetime(train["pickup_datetime"]).apply(lambda x: x.strftime("%Y-%m-%d"))

train["extreme_weather"] = train["y-m-d"].apply(lambda x: 1 if x in weather_event else 0)

In [18]:
taxi["weather_event"] = train["extreme_weather"]

In [21]:
#trip_duration 위치 변경
trip_duration = taxi["trip_duration"]
del taxi["trip_duration"]
taxi["trip_duration"] = trip_duration

## 3.1 명확한 이상치 제거

### 3.1.1 종속변수 기준, 명확한 이상치
150만초( > 17일 데이터) 4개 확인

In [24]:
# > 1,500,000 second
taxi[taxi["trip_duration"] > 1500000]

,id,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,dist,pick_date,pick_time,drop_time,holiday,holiday_Fri,weekday,pickup_hour,dropoff_hour,weather_event,trip_duration
355003,1864733,1,1,-73.789650,40.643559,-73.956810,40.773087,0,20.136014,4,1181,40117,0,0,1,0,11,0,1939736
680594,369307,1,2,-73.921677,40.735252,-73.984749,40.759979,0,5.978731,43,81479,57457,1,1,5,22,15,0,2049578
924150,1325766,1,1,-73.983788,40.742325,-73.985489,40.727676,0,1.634101,4,22454,3666,0,0,1,6,1,0,2227612
978383,53347,1,1,-73.783905,40.648632,-73.978271,40.750202,0,19.888167,43,82011,65893,1,1,5,22,18,0,3526282


In [27]:
#이동거리 확인
over_million = taxi[taxi["trip_duration"] > 1500000]
pick_dropoff(over_million)

** 탑승시간 대비 이동거리가 너무 짧음 -> 오류로 판단 **

In [28]:
#데이터 삭제
taxi = taxi[taxi["trip_duration"] < 1500000]

### 3.1.2 이동거리 기준, 명확한 이상치 제거
직선 이동거리가 0인 데이터 확인

In [29]:
print("이동거리가 0인 데이터 개수 : ", len(taxi[taxi["dist"] == 0]))

이동거리가 0인 데이터 개수 :  5897


** 택시 탑승 후 이동 거리는 0일 수 없음 -> 오류로 판단 **

In [30]:
taxi = taxi[taxi["dist"] != 0]
taxi.reset_index(drop = True)

,id,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,dist,pick_date,pick_time,drop_time,holiday,holiday_Fri,weekday,pickup_hour,dropoff_hour,weather_event,trip_duration
0,2875421,2,1,-73.982155,40.767937,-73.964630,40.765602,0,1.497580,73,62694,63149,0,0,0,17,17,1,455
1,2377394,1,1,-73.980415,40.738564,-73.999481,40.731152,0,1.804374,163,2614,3277,1,1,6,0,0,0,663
2,3858529,2,1,-73.979027,40.763939,-74.005333,40.710087,0,6.381090,18,41723,43847,0,0,1,11,12,0,2124
3,3504673,2,1,-74.010040,40.719971,-74.012268,40.706718,0,1.484566,96,70350,70779,0,0,2,19,19,0,429
4,2181028,2,1,-73.973053,40.793209,-73.972923,40.782520,0,1.187842,85,48654,49089,1,1,5,13,13,0,435
5,801584,2,6,-73.982857,40.742195,-73.992081,40.749184,0,1.098252,29,79299,79742,1,1,5,22,22,0,443
6,1813257,1,4,-73.969017,40.757839,-73.957405,40.765896,0,1.325446,168,81298,81639,0,1,4,22,22,0,341
7,1324603,2,1,-73.969276,40.797779,-73.922470,40.760559,0,5.711393,141,28497,30048,1,1,5,7,8,0,1551
8,1301050,1,1,-73.999481,40.738400,-73.985786,40.732815,0,1.309531,147,83542,83797,0,1,4,23,23,0,255
9,12891,2,1,-73.981049,40.744339,-73.973000,40.789989,0,5.117946,69,78300,79525,0,0,3,21,22,0,1225


## 3.2 회귀 분석(OLS)

In [31]:
features = " + ".join(taxi.columns)

In [32]:
features = 'id + vendor_id + passenger_count + pickup_longitude + pickup_latitude\
+ dropoff_longitude + dropoff_latitude + store_and_fwd_flag + dist + pick_date + pick_time \
+ drop_time + holiday + holiday_Fri + weekday + pickup_hour + dropoff_hour + weather_event'

model =  sm.OLS.from_formula("trip_duration ~ {}".format(features), data = taxi)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     2883.
Date:                Wed, 14 Mar 2018   Prob (F-statistic):               0.00
Time:                        01:24:04   Log-Likelihood:            -1.3750e+07
No. Observations:             1452743   AIC:                         2.750e+07
Df Residuals:                 1452724   BIC:                         2.750e+07
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept           1.075e+05   5540.237     19.411      0.000    9.67e+04    1.18e+05
id                 -2.152e-06   2.24e-06     -0.960      0.337   -6.55e-06    2.24e-06
vendor_id            209.7702      5.435     38.595      0.000     199.117     220.423
passenger_count        8.8241      2.058      4.288      0.000       4.791      12.858
pickup_longitude    -338.9332     61.349     -5.525      0.000    -459.175    -218.691
pickup_latitude    -1135.0149     95.116    -11.933      0.000   -1321.439    -948.591
dropoff_longitude    634.2822     60.294     10.520      0.000     516.108     752.457
dropoff_latitude    -973.1739     85.007    -11.448      0.000   -1139.784    -806.564
store_and_fwd_flag    67.6123     35.152      1.923      0.054      -1.284     136.509
dist                 108.5636      0.650    166.903      0.000     107.289     109.839
pick_date              0.6215      0.051     12.196      0.000       0.522       0.721
pick_time             -0.0279      0.002    -11.601      0.000      -0.033      -0.023
drop_time              0.2099      0.002    119.679      0.000       0.206       0.213
holiday             -121.0494      8.839    -13.694      0.000    -138.374    -103.724
holiday_Fri          -25.0056      9.681     -2.583      0.010     -43.980      -6.031
weekday               21.5642      2.484      8.683      0.000      16.697      26.432
pickup_hour           97.3797      8.720     11.167      0.000      80.288     114.471
dropoff_hour        -748.4670      6.364   -117.618      0.000    -760.939    -735.995
weather_event        -26.8574      9.345     -2.874      0.004     -45.173      -8.541
==============================================================================
Omnibus:                  3528337.471   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      30043543440.966
Skew:                          25.979   Prob(JB):                         0.00
Kurtosis:                     705.591   Cond. No.                     4.94e+09
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.94e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 3.2.1 ANOVA test를 통한 각 변수의 영향력 파악

In [36]:
sm.stats.anova_lm(result, typ = 2)

,sum_sq,df,F,PR(>F)
id,8.970983e+06,1.0,0.921581,3.370603e-01
vendor_id,1.449971e+10,1.0,1489.542350,0.000000e+00
passenger_count,1.789650e+08,1.0,18.384915,1.804928e-05
pickup_longitude,2.971127e+08,1.0,30.522121,3.301375e-08
pickup_latitude,1.386124e+09,1.0,142.395317,7.997982e-33
dropoff_longitude,1.077257e+09,1.0,110.665627,7.019244e-26
dropoff_latitude,1.275795e+09,1.0,131.061259,2.408034e-30
store_and_fwd_flag,3.601342e+07,1.0,3.699626,5.442485e-02
dist,2.711654e+11,1.0,27856.575959,0.000000e+00
pick_date,1.447861e+09,1.0,148.737449,3.285537e-34


> ** F 검정의 p-value가 상대적으로 큰 id, store_and_fwd_flag, holiday_Fri 제외 **

In [37]:
del taxi["id"]
del taxi["store_and_fwd_flag"]
del taxi["holiday_Fri"]

### 3.2.2 VIF를 통한 다중공선성 파악
> 의존적 변수 제거

In [38]:
x = taxi.iloc[ : , : -1]
y = taxi.trip_duration

In [39]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
vif["features"] = x.columns
vif

,VIF Factor,features
0,1.141176e+01,vendor_id
1,2.842221e+00,passenger_count
2,2.944884e+06,pickup_longitude
3,1.699793e+06,pickup_latitude
4,2.732942e+06,dropoff_longitude
5,1.478356e+06,dropoff_latitude
6,1.717013e+00,dist
7,4.225497e+00,pick_date
8,2.676879e+03,pick_time
9,1.434614e+03,drop_time


** VIF Factor가 상대적으로 높아 다중공선성이 높다고 판단되는 4개의 변수 제외 **
> pickup_longitude, pickup_latitdue, dropoff_longitude, dropoff_latitude

In [40]:
del taxi["pickup_longitude"]
del taxi["pickup_latitude"]
del taxi["dropoff_longitude"]
del taxi["dropoff_latitude"]

In [41]:
x = taxi.iloc[ : , : -1]
y = taxi.trip_duration

In [42]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
vif["features"] = x.columns
vif

,VIF Factor,features
0,8.081014,vendor_id
1,2.830271,passenger_count
2,1.610560,dist
3,3.665822,pick_date
4,2425.501595,pick_time
5,1371.278902,drop_time
6,3.129831,holiday
7,6.846964,weekday
8,2344.461206,pickup_hour
9,1320.210727,dropoff_hour


In [46]:
features = 'vendor_id + passenger_count + dist + pick_date + holiday + weekday + pickup_hour + dropoff_hour + weather_event'

model =  sm.OLS.from_formula("trip_duration ~ {}".format(features), data = taxi)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     4073.
Date:                Wed, 14 Mar 2018   Prob (F-statistic):               0.00
Time:                        01:45:23   Log-Likelihood:            -1.3757e+07
No. Observations:             1452743   AIC:                         2.751e+07
Df Residuals:                 1452733   BIC:                         2.751e+07
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         106.8609     12.552      8.514      0.000      82.260     131.462
vendor_id         212.6652      5.446     39.048      0.000     201.991     223.340
passenger_count     8.9673      2.068      4.335      0.000       4.913      13.021
dist              112.0129      0.606    184.890      0.000     110.825     113.200
pick_date           0.6218      0.051     12.143      0.000       0.521       0.722
holiday          -126.8759      8.324    -15.242      0.000    -143.191    -110.561
weekday            18.0525      1.994      9.052      0.000      14.144      21.961
pickup_hour         2.8869      1.137      2.539      0.011       0.658       5.116
dropoff_hour        0.7842      1.123      0.698      0.485      -1.416       2.985
weather_event     -27.3577      9.374     -2.919      0.004     -45.730      -8.986
==============================================================================
Omnibus:                  3525515.524   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      29682445608.684
Skew:                          25.933   Prob(JB):                         0.00
Kurtosis:                     701.339   Cond. No.                         544.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [50]:
sm.stats.anova_lm(result, typ = 2)

,sum_sq,df,F,PR(>F)
vendor_id,1.499470e+10,1.0,1524.804065,0.000000e+00
passenger_count,1.848345e+08,1.0,18.795734,1.455019e-05
dist,3.362164e+11,1.0,34189.680122,0.000000e+00
pick_date,1.449397e+09,1.0,147.388489,6.478162e-34
holiday,2.284334e+09,1.0,232.292793,1.902831e-52
weekday,8.037947e+08,1.0,81.737494,1.556001e-19
pickup_hour,7.756039e+08,1.0,78.870783,6.638101e-19
weather_event,8.370765e+07,1.0,8.512190,3.527806e-03
Residual,1.428598e+13,1452734.0,NaN,NaN


> ** F 검정의 p-value가 상대적으로 낮은  dropoff_hour 제거 **

In [48]:
del taxi["dropoff_hour"]

In [49]:
features = 'vendor_id + passenger_count + dist + pick_date + \
holiday + weekday + pickup_hour + weather_event'


model =  sm.OLS.from_formula("trip_duration ~ {}".format(features), data = taxi)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     4582.
Date:                Wed, 14 Mar 2018   Prob (F-statistic):               0.00
Time:                        01:46:46   Log-Likelihood:            -1.3757e+07
No. Observations:             1452743   AIC:                         2.751e+07
Df Residuals:                 1452734   BIC:                         2.751e+07
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         107.5428     12.514      8.594      0.000      83.016     132.069
vendor_id         212.6714      5.446     39.049      0.000     201.997     223.346
passenger_count     8.9675      2.068      4.335      0.000       4.913      13.022
dist              112.0057      0.606    184.905      0.000     110.818     113.193
pick_date           0.6216      0.051     12.140      0.000       0.521       0.722
holiday          -126.8705      8.324    -15.241      0.000    -143.186    -110.555
weekday            18.0269      1.994      9.041      0.000      14.119      21.935
pickup_hour         3.6282      0.409      8.881      0.000       2.827       4.429
weather_event     -27.3483      9.374     -2.918      0.004     -45.720      -8.976
==============================================================================
Omnibus:                  3525507.334   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      29681996067.604
Skew:                          25.933   Prob(JB):                         0.00
Kurtosis:                     701.334   Cond. No.                         539.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [51]:
x = taxi.iloc[ : , : -1]
y = taxi.trip_duration

In [52]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
vif["features"] = x.columns
vif

,VIF Factor,features
0,7.933806,vendor_id
1,2.829794,passenger_count
2,1.608033,dist
3,3.644127,pick_date
4,2373.609792,pick_time
5,43.653727,drop_time
6,3.127913,holiday
7,6.810641,weekday
8,2284.902328,pickup_hour
9,1.139806,weather_event


> **VIF Factor가 상대적으로 높은 pick_time, pick_hour 제거 **

(결과적으로 추가로 입력했던 pick_hour, dropoff_hour 모두 제거, 넣어주지 말 것)

In [53]:
del taxi["pick_time"]
del taxi["pickup_hour"]

In [55]:
features = 'vendor_id + passenger_count + dist + pick_date + \
holiday + weekday + weather_event'

model =  sm.OLS.from_formula("trip_duration ~ {}".format(features), data = taxi)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     5225.
Date:                Wed, 14 Mar 2018   Prob (F-statistic):               0.00
Time:                        01:49:12   Log-Likelihood:            -1.3757e+07
No. Observations:             1452743   AIC:                         2.751e+07
Df Residuals:                 1452735   BIC:                         2.751e+07
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         159.4004     11.068     14.402      0.000     137.707     181.094
vendor_id         212.9765      5.446     39.104      0.000     202.302     223.651
passenger_count     9.1442      2.068      4.421      0.000       5.090      13.198
dist              111.9254      0.606    184.788      0.000     110.738     113.113
pick_date           0.6174      0.051     12.058      0.000       0.517       0.718
holiday          -129.9726      8.317    -15.627      0.000    -146.274    -113.671
weekday            17.5289      1.993      8.794      0.000      13.622      21.436
weather_event     -28.5378      9.373     -3.045      0.002     -46.908     -10.167
==============================================================================
Omnibus:                  3525417.391   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      29675567626.173
Skew:                          25.931   Prob(JB):                         0.00
Kurtosis:                     701.258   Cond. No.                         494.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 종속변수 분포 확인 

In [ ]:
sp.stats.probplot(np.log(taxi["trip_duration"]), plot = plt)
plt.show()

> ** log를 씌워주며 선형성이 증가함을 확인 **

In [ ]:
#종속변수 trip_duration에 np.log 취해줌
features = 'vendor_id + scale(dist) + pick_date + holiday + weekday + pickup_hour'

model =  sm.OLS.from_formula("np.log1p(trip_duration) ~ {}".format(features), data = taxi)
result = model.fit()
result.summary()

> ** R square 증가 **

--------------------------------------------------------

(한준 매니저님의 깃헙 참고)

### 실수값 변수의 다항회귀 차수 결정

$ y = vendor id + distance + distance^2 + pick date + holiday + weekday$

In [ ]:
features = 'vendor_id + dist+ I(dist ** 2) + weekday + pick_date + holiday + pickup_hour'

model =  sm.OLS.from_formula("np.log1p(trip_duration) ~ {}".format(features), data = taxi)
result = model.fit()
result.summary()

In [ ]:
sm.stats.anova_lm(result, typ = 2)

$ y = vendor id + distance + distance^2+ distance^3 + pick date + holiday + weekday$

In [ ]:
features = 'vendor_id + dist+ I(dist ** 2)+ I(dist ** 3) + weekday + pick_date + holiday + pickup_hour'

model =  sm.OLS.from_formula("np.log1p(trip_duration) ~ {}".format(features), data = taxi)
result = model.fit()
result.summary()

In [ ]:
sm.stats.anova_lm(result, typ = 2)

$ y = vendor id + distance + distance^2+ distance^3+ distance^4 + pick date + holiday + weekday$

In [ ]:
features = 'scale(dist) + scale(I(dist ** 2)) + scale(I(dist ** 3))+ scale(I(dist ** 4))\
+ weekday + pick_date + holiday + pickup_hour'

model =  sm.OLS.from_formula("np.log1p(trip_duration) ~ {}".format(features), data = taxi)
result = model.fit()
result.summary()

In [ ]:
sm.stats.anova_lm(result, typ = 2)

In [ ]:
test = erase_outlier_np(result, taxi, total_feature = result.df_model)

In [ ]:
len(test)

In [ ]:
features = 'dist + I(dist ** 2)\
+ weekday + pick_date + holiday + pickup_hour'

model =  sm.OLS.from_formula("np.log1p(trip_duration) ~ {}".format(features), data = test)
result = model.fit()
result.summary()

In [ ]:
sp.stats.probplot(result.resid, plot = plt)
plt.show()

(kaggle에서 어떤 사람이 택시기사와 얘기한 후, 택시기사들이 가솔린 가격에 따라 
운전속도를 조절한다는 이야기를 보고 반영할까 말까 고민 중, 팀원들과 상의해볼것)

In [ ]:
oil = pd.read_csv("gasoline.csv")

In [ ]:
pd.to_datetime(oil["Date"])